# ML Modeling Pipeline: Strategic Voucher Targeting

This notebook implements an end-to-end modeling workflow for maximizing expected revenue from sending €5 vouchers to customers after their first purchase.

**Business Objective**: Optimize the decision to send vouchers to maximize net revenue.

**Key Decision Rule**: Send voucher when predicted probability of reorder (target90=1) is **below** a threshold, i.e., send to customers predicted as **not reordering**.


## 1. Imports and Config


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.calibration import calibration_curve
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay
)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Optuna for hyperparameter tuning
try:
    import optuna
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False
    print("Warning: Optuna not available. Hyperparameter tuning will be limited.")

# XGBoost
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("Warning: XGBoost not available. Using HistGradientBoostingClassifier as fallback.")

# Joblib for model persistence
import joblib

# Set random seeds for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Imports completed successfully.")
print(f"Optuna available: {OPTUNA_AVAILABLE}")
print(f"XGBoost available: {XGBOOST_AVAILABLE}")


## 2. Data Loading / Inputs

Load the preprocessed datasets. If running this notebook standalone, we'll need to run the preprocessing pipeline first. Otherwise, we assume `X_train_df`, `X_val_df`, `X_test_df`, `y_train`, and `y_val` are already available in memory.


In [ ]:
# Check if preprocessed data exists in memory
try:
    # Try to access the preprocessed data
    _ = X_train_df.shape
    _ = X_val_df.shape
    _ = X_test_df.shape
    _ = y_train.shape
    _ = y_val.shape
    print("Using preprocessed data from memory.")
    data_loaded = True
except NameError:
    print("Preprocessed data not found in memory. Loading and preprocessing from raw files...")
    data_loaded = False


In [ ]:
if not data_loaded:
    # Load raw data
    train = pd.read_csv('train.csv', sep=';', low_memory=False)
    test = pd.read_csv('test.csv', sep=';', low_memory=False)
    
    # Feature engineering function (from preprocessing notebook)
    def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
        """Apply feature engineering steps."""
        out = df.copy()
        
        # Drop columns
        out.drop(columns=["customernumber", "points", "delivpostcode"], inplace=True, errors="ignore")
        
        # Advertising indicator
        if "advertisingdatacode" in out.columns:
            out["has_ad_code"] = out["advertisingdatacode"].notna().astype(int)
            out.drop(columns=["advertisingdatacode"], inplace=True)
        
        # Parse datetime columns
        for c in ["date", "datecreated", "deliverydatepromised", "deliverydatereal"]:
            if c in out.columns:
                out[c] = pd.to_datetime(out[c], errors="coerce")
        
        # Engineered time features
        out["account_age_days"] = (out["date"] - out["datecreated"]).dt.days
        out["order_weekday"] = out["date"].dt.weekday
        out["order_month"] = out["date"].dt.month
        out["promised_delivery_weekday"] = out["deliverydatepromised"].dt.weekday
        out["promised_delivery_month"] = out["deliverydatepromised"].dt.month
        out["delivery_difference"] = (out["deliverydatereal"] - out["deliverydatepromised"]).dt.days
        
        # Drop raw datetime columns
        out.drop(columns=["date", "datecreated", "deliverydatepromised", "deliverydatereal"], 
                inplace=True, errors="ignore")
        
        # Product diversity
        w_cols = [f"w{i}" for i in range(11)]
        existing_w = [c for c in w_cols if c in out.columns]
        out["product_diversity"] = (out[existing_w] > 0).sum(axis=1)
        
        # Drop invoicepostcode
        out.drop(columns=["invoicepostcode"], inplace=True, errors="ignore")
        
        return out
    
    # Apply feature engineering
    train_fe = engineer_features(train)
    test_fe = engineer_features(test)
    
    # Split train/val
    from sklearn.model_selection import train_test_split
    X = train_fe.drop(columns=["target90"])
    y = train_fe["target90"]
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
    )
    
    # Preprocessing pipeline
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline as SKPipeline
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.impute import SimpleImputer
    
    w_cols = [f"w{i}" for i in range(11)]
    cat_cols = [
        "salutation", "title", "domain", "newsletter", "model",
        "paymenttype", "deliverytype", "voucher", "gift", "entry", "shippingcosts"
    ]
    num_cols = [
        "case", "numberitems", "weight", "remi", "cancel", "used",
        "has_ad_code", "account_age_days", "order_weekday", "order_month",
        "promised_delivery_weekday", "promised_delivery_month",
        "delivery_difference", "product_diversity", *w_cols
    ]
    
    numeric_pipe = SKPipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
    categorical_pipe = SKPipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])
    
    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", categorical_pipe, cat_cols),
        ],
        remainder="drop"
    )
    
    # Fit and transform
    X_train_t = preprocess.fit_transform(X_train)
    X_val_t = preprocess.transform(X_val)
    X_test_t = preprocess.transform(test_fe)
    
    # Get feature names
    num_names = num_cols
    ohe = preprocess.named_transformers_["cat"].named_steps["ohe"]
    cat_names = ohe.get_feature_names_out(cat_cols).tolist()
    feature_names = num_names + cat_names
    
    # Create DataFrames
    X_train_df = pd.DataFrame(X_train_t, columns=feature_names, index=X_train.index)
    X_val_df = pd.DataFrame(X_val_t, columns=feature_names, index=X_val.index)
    X_test_df = pd.DataFrame(X_test_t, columns=feature_names, index=test_fe.index)
    
    print("Data loaded and preprocessed successfully.")


In [ ]:
# Quality checks
assert X_train_df.columns.equals(X_val_df.columns), "Train and validation columns don't match"
assert X_val_df.columns.equals(X_test_df.columns), "Validation and test columns don't match"
assert X_train_df.isna().sum().sum() == 0, "NaNs found in X_train_df"
assert X_val_df.isna().sum().sum() == 0, "NaNs found in X_val_df"
assert X_test_df.isna().sum().sum() == 0, "NaNs found in X_test_df"

print(f"Train shape: {X_train_df.shape}")
print(f"Validation shape: {X_val_df.shape}")
print(f"Test shape: {X_test_df.shape}")
print(f"Target distribution (train): {y_train.value_counts().to_dict()}")
print(f"Target distribution (val): {y_val.value_counts().to_dict()}")
print(f"Positive class rate (train): {y_train.mean():.4f}")
print(f"Positive class rate (val): {y_val.mean():.4f}")


## 3. Business Objective and Revenue Metric

**Business Payoff Structure:**
- Send voucher to customer who would **NOT reorder** (true class 0): +€1.25 expected uplift
- Send voucher to customer who **WOULD reorder anyway** (true class 1): -€5 loss
- Do not send voucher: €0 impact

**Decision Rule:**
- We send a voucher when predicted probability of reorder (P(reorder=1)) is **below** a threshold `t`
- Equivalently: send voucher when predicted as "will NOT reorder" (class 0)

**Why optimize threshold?** The default 0.5 threshold maximizes accuracy but not revenue. Since the cost of sending a voucher to a reorderer (-€5) is much higher than the benefit of sending to a non-reorderer (+€1.25), we need to find the threshold that maximizes expected revenue, not accuracy.

**Why use revenue metric over accuracy?** Accuracy treats all misclassifications equally, but in this business context, the costs and benefits are asymmetric. A revenue-based metric directly optimizes for the business objective.


In [ ]:
def calculate_revenue(y_true, y_pred_proba, threshold):
    """
    Calculate net revenue based on predicted probabilities and threshold.
    
    Decision rule: send_voucher = 1 if p < threshold else 0
    (i.e., send voucher when predicted probability of reorder is LOW)
    
    Revenue calculation:
    - If send_voucher==1 and y_true==0 (would not reorder) => +1.25
    - If send_voucher==1 and y_true==1 (would reorder anyway) => -5
    - Else (do not send) => 0
    
    Parameters:
    -----------
    y_true : array-like
        True labels (1 = reorder, 0 = no reorder)
    y_pred_proba : array-like
        Predicted probabilities of class 1 (reorder)
    threshold : float
        Threshold for decision (send voucher if p < threshold)
    
    Returns:
    --------
    total_revenue : float
        Total revenue across all customers
    avg_revenue : float
        Average revenue per customer
    send_decisions : array
        Binary array indicating send (1) or not send (0)
    """
    y_true = np.array(y_true)
    y_pred_proba = np.array(y_pred_proba)
    
    # Decision: send voucher if probability of reorder is BELOW threshold
    send_voucher = (y_pred_proba < threshold).astype(int)
    
    # Calculate revenue
    revenue = np.zeros(len(y_true))
    
    # Send voucher to non-reorderers: +1.25
    revenue[(send_voucher == 1) & (y_true == 0)] = 1.25
    
    # Send voucher to reorderers: -5
    revenue[(send_voucher == 1) & (y_true == 1)] = -5.0
    
    # Do not send: 0 (already initialized)
    
    total_revenue = revenue.sum()
    avg_revenue = total_revenue / len(y_true)
    
    return total_revenue, avg_revenue, send_voucher


def optimize_threshold(y_true, y_pred_proba, threshold_range=None):
    """
    Find the threshold that maximizes revenue on validation set.
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred_proba : array-like
        Predicted probabilities of class 1
    threshold_range : array-like, optional
        Range of thresholds to search. Default: np.linspace(0.01, 0.99, 99)
    
    Returns:
    --------
    best_threshold : float
        Threshold that maximizes revenue
    best_revenue : float
        Maximum revenue achieved
    results : DataFrame
        DataFrame with threshold, revenue pairs
    """
    if threshold_range is None:
        threshold_range = np.linspace(0.01, 0.99, 99)
    
    results = []
    
    for t in threshold_range:
        total_rev, avg_rev, _ = calculate_revenue(y_true, y_pred_proba, t)
        results.append({
            'threshold': t,
            'total_revenue': total_rev,
            'avg_revenue': avg_rev
        })
    
    results_df = pd.DataFrame(results)
    best_idx = results_df['total_revenue'].idxmax()
    best_threshold = results_df.loc[best_idx, 'threshold']
    best_revenue = results_df.loc[best_idx, 'total_revenue']
    
    return best_threshold, best_revenue, results_df


# Test the revenue function with a simple example
print("Revenue function test:")
test_y = np.array([0, 0, 1, 1])
test_proba = np.array([0.1, 0.9, 0.1, 0.9])  # Low prob = send voucher
test_thresh = 0.5
total, avg, decisions = calculate_revenue(test_y, test_proba, test_thresh)
print(f"  Test case: y_true={test_y}, proba={test_proba}, threshold={test_thresh}")
print(f"  Send decisions: {decisions}")
print(f"  Total revenue: {total:.2f}, Avg revenue: {avg:.2f}")
print("\nRevenue function validated.")


In [ ]:
# Baseline 1: Always send voucher
def baseline_always_send(y_true):
    """Calculate revenue if we send voucher to everyone."""
    n = len(y_true)
    n_non_reorder = (y_true == 0).sum()
    n_reorder = (y_true == 1).sum()
    revenue = n_non_reorder * 1.25 + n_reorder * (-5.0)
    return revenue, revenue / n


# Baseline 2: Never send voucher
def baseline_never_send(y_true):
    """Calculate revenue if we never send voucher."""
    return 0.0, 0.0


# Baseline 3: Random (50% send rate)
def baseline_random(y_true, random_state=RANDOM_STATE):
    """Calculate revenue with random 50% send rate."""
    np.random.seed(random_state)
    n = len(y_true)
    send_decisions = np.random.binomial(1, 0.5, n)
    revenue = np.zeros(n)
    revenue[(send_decisions == 1) & (y_true == 0)] = 1.25
    revenue[(send_decisions == 1) & (y_true == 1)] = -5.0
    return revenue.sum(), revenue.sum() / n


# Calculate baselines on validation set
val_baselines = {
    'Always Send': baseline_always_send(y_val),
    'Never Send': baseline_never_send(y_val),
    'Random (50%)': baseline_random(y_val)
}

print("Baseline Performance on Validation Set:")
print("=" * 50)
for name, (total, avg) in val_baselines.items():
    print(f"{name:20s}: Total Revenue = €{total:8.2f}, Avg Revenue = €{avg:6.4f}")
print("=" * 50)


## 5. Model Training + Tuning

Train multiple models and tune hyperparameters using Optuna (for at least one model).


In [ ]:
# Storage for models and results
models = {}
results = []

def plot_confusion_matrix(y_true, y_pred, title, ax=None):
    """
    Plot confusion matrix using ConfusionMatrixDisplay.
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels
    title : str
        Title for the plot
    ax : matplotlib.axes, optional
        Axes to plot on. If None, creates new figure.
    """
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Reorder', 'Reorder'])
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    disp.plot(ax=ax, cmap='Blues', values_format='d')
    ax.set_title(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    return ax


def evaluate_model(model, model_name, X_train, y_train, X_val, y_val):
    """
    Train a model and evaluate it on validation set using revenue-optimal threshold.
    Returns dictionary with all metrics and predictions.
    
    CRITICAL: All classification metrics (accuracy, precision, recall, F1, confusion matrix)
    are computed using y_pred_optimal = (y_pred_proba < best_threshold), which is the
    revenue-optimal decision rule, NOT the default model.predict().
    """
    # Train
    model.fit(X_train, y_train)
    
    # Probabilities (handle models without predict_proba)
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_val)[:, 1]
    elif hasattr(model, 'decision_function'):
        # For SVM-like models, use decision function and sigmoid
        decision = model.decision_function(X_val)
        y_pred_proba = 1 / (1 + np.exp(-decision))  # Sigmoid
    else:
        # Fallback: use predict and convert to probabilities
        y_pred = model.predict(X_val)
        y_pred_proba = y_pred.astype(float)
    
    # Ensure probabilities are in [0, 1]
    y_pred_proba = np.clip(y_pred_proba, 0, 1)
    
    # Assert probabilities are valid
    assert np.all((y_pred_proba >= 0) & (y_pred_proba <= 1)), f"Invalid probabilities for {model_name}"
    
    # Threshold optimization for revenue
    best_threshold, best_revenue, threshold_results = optimize_threshold(y_val, y_pred_proba)
    
    # Revenue-optimal decision: send voucher if p < threshold
    y_pred_optimal = (y_pred_proba < best_threshold).astype(int)
    
    # Standard metrics computed on revenue-optimal decisions
    accuracy = accuracy_score(y_val, y_pred_optimal)
    precision = precision_score(y_val, y_pred_optimal, zero_division=0)
    recall = recall_score(y_val, y_pred_optimal, zero_division=0)
    f1 = f1_score(y_val, y_pred_optimal, zero_division=0)
    
    # ROC-AUC computed on probabilities (not thresholded)
    try:
        roc_auc = roc_auc_score(y_val, y_pred_proba)
    except:
        roc_auc = np.nan
    
    # Confusion matrix using revenue-optimal decisions
    cm_optimal = confusion_matrix(y_val, y_pred_optimal)
    
    # Classification report text
    report_text = classification_report(y_val, y_pred_optimal, target_names=['No Reorder', 'Reorder'])
    
    # Revenue at optimal threshold
    total_revenue, avg_revenue, _ = calculate_revenue(y_val, y_pred_proba, best_threshold)
    
    result = {
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'best_threshold': best_threshold,
        'total_revenue': total_revenue,
        'avg_revenue': avg_revenue,
        'confusion_matrix': cm_optimal,  # Using revenue-optimal decisions
        'y_pred_proba': y_pred_proba,
        'y_pred_optimal': y_pred_optimal,  # Revenue-optimal binary predictions
        'threshold_results': threshold_results,
        'classification_report': report_text
    }
    
    return result, model


### 5.1 Logistic Regression (Baseline)


In [ ]:
print("Training Logistic Regression...")
print("=" * 80)

# Light tuning for Logistic Regression
print("Performing light hyperparameter tuning...")
lr_param_grid = {
    'C': np.logspace(-3, 2, 6),
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}
lr_base = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000, class_weight='balanced')
lr_search = RandomizedSearchCV(
    lr_base, lr_param_grid, n_iter=12, cv=3, scoring='roc_auc',
    random_state=RANDOM_STATE, n_jobs=-1, verbose=0
)
lr_search.fit(X_train_df, y_train)
lr = lr_search.best_estimator_
print(f"Best parameters: {lr_search.best_params_}")

result_lr, model_lr = evaluate_model(lr, 'Logistic Regression', X_train_df, y_train, X_val_df, y_val)
models['Logistic Regression'] = model_lr
results.append(result_lr)

print(f"\nValidation Metrics (at threshold={result_lr['best_threshold']:.4f}):")
print(f"  Accuracy: {result_lr['accuracy']:.4f}")
print(f"  Precision: {result_lr['precision']:.4f}")
print(f"  Recall: {result_lr['recall']:.4f}")
print(f"  F1: {result_lr['f1']:.4f}")
print(f"  ROC-AUC: {result_lr['roc_auc']:.4f}")
print(f"  Total Revenue: €{result_lr['total_revenue']:.2f}")
print(f"  Avg Revenue: €{result_lr['avg_revenue']:.4f}")

# Confusion matrix plot
plot_confusion_matrix(
    y_val, result_lr['y_pred_optimal'],
    f"Logistic Regression (threshold={result_lr['best_threshold']:.4f})"
)
plt.show()

# Classification report
print("\nClassification Report:")
print(result_lr['classification_report'])


In [ ]:
print("Training Random Forest...")
print("=" * 80)
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

result_rf, model_rf = evaluate_model(rf, 'Random Forest', X_train_df, y_train, X_val_df, y_val)
models['Random Forest'] = model_rf
results.append(result_rf)

print(f"\nValidation Metrics (at threshold={result_rf['best_threshold']:.4f}):")
print(f"  Accuracy: {result_rf['accuracy']:.4f}")
print(f"  Precision: {result_rf['precision']:.4f}")
print(f"  Recall: {result_rf['recall']:.4f}")
print(f"  F1: {result_rf['f1']:.4f}")
print(f"  ROC-AUC: {result_rf['roc_auc']:.4f}")
print(f"  Total Revenue: €{result_rf['total_revenue']:.2f}")
print(f"  Avg Revenue: €{result_rf['avg_revenue']:.4f}")

# Confusion matrix plot
plot_confusion_matrix(
    y_val, result_rf['y_pred_optimal'],
    f"Random Forest (threshold={result_rf['best_threshold']:.4f})"
)
plt.show()

# Classification report
print("\nClassification Report:")
print(result_rf['classification_report'])


### 5.3 Gradient Boosting


In [ ]:
print("Training Gradient Boosting...")
print("=" * 80)
gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=RANDOM_STATE
)

result_gb, model_gb = evaluate_model(gb, 'Gradient Boosting', X_train_df, y_train, X_val_df, y_val)
models['Gradient Boosting'] = model_gb
results.append(result_gb)

print(f"\nValidation Metrics (at threshold={result_gb['best_threshold']:.4f}):")
print(f"  Accuracy: {result_gb['accuracy']:.4f}")
print(f"  Precision: {result_gb['precision']:.4f}")
print(f"  Recall: {result_gb['recall']:.4f}")
print(f"  F1: {result_gb['f1']:.4f}")
print(f"  ROC-AUC: {result_gb['roc_auc']:.4f}")
print(f"  Total Revenue: €{result_gb['total_revenue']:.2f}")
print(f"  Avg Revenue: €{result_gb['avg_revenue']:.4f}")

# Confusion matrix plot
plot_confusion_matrix(
    y_val, result_gb['y_pred_optimal'],
    f"Gradient Boosting (threshold={result_gb['best_threshold']:.4f})"
)
plt.show()

# Classification report
print("\nClassification Report:")
print(result_gb['classification_report'])


### 5.4 AdaBoost


In [ ]:
print("Training AdaBoost...")
print("=" * 80)

# Light tuning for AdaBoost
print("Performing light hyperparameter tuning...")
ada_param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.5, 1.0, 1.5]
}
ada_base = AdaBoostClassifier(random_state=RANDOM_STATE)
ada_search = RandomizedSearchCV(
    ada_base, ada_param_grid, n_iter=9, cv=3, scoring='roc_auc',
    random_state=RANDOM_STATE, n_jobs=-1, verbose=0
)
ada_search.fit(X_train_df, y_train)
ada = ada_search.best_estimator_
print(f"Best parameters: {ada_search.best_params_}")

result_ada, model_ada = evaluate_model(ada, 'AdaBoost', X_train_df, y_train, X_val_df, y_val)
models['AdaBoost'] = model_ada
results.append(result_ada)

print(f"\nValidation Metrics (at threshold={result_ada['best_threshold']:.4f}):")
print(f"  Accuracy: {result_ada['accuracy']:.4f}")
print(f"  Precision: {result_ada['precision']:.4f}")
print(f"  Recall: {result_ada['recall']:.4f}")
print(f"  F1: {result_ada['f1']:.4f}")
print(f"  ROC-AUC: {result_ada['roc_auc']:.4f}")
print(f"  Total Revenue: €{result_ada['total_revenue']:.2f}")
print(f"  Avg Revenue: €{result_ada['avg_revenue']:.4f}")

# Confusion matrix plot
plot_confusion_matrix(
    y_val, result_ada['y_pred_optimal'],
    f"AdaBoost (threshold={result_ada['best_threshold']:.4f})"
)
plt.show()

# Classification report
print("\nClassification Report:")
print(result_ada['classification_report'])


### 5.5 K-Nearest Neighbors (with scaling)


In [ ]:
print("Training K-Nearest Neighbors (with StandardScaler)...")
print("=" * 80)

# Light tuning for KNN
print("Performing light hyperparameter tuning...")
knn_param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]
}
knn_base = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])
knn_search = RandomizedSearchCV(
    knn_base, knn_param_grid, n_iter=20, cv=3, scoring='roc_auc',
    random_state=RANDOM_STATE, n_jobs=-1, verbose=0
)
knn_search.fit(X_train_df, y_train)
knn_pipeline = knn_search.best_estimator_
print(f"Best parameters: {knn_search.best_params_}")

result_knn, model_knn = evaluate_model(knn_pipeline, 'KNN', X_train_df, y_train, X_val_df, y_val)
models['KNN'] = model_knn
results.append(result_knn)

print(f"\nValidation Metrics (at threshold={result_knn['best_threshold']:.4f}):")
print(f"  Accuracy: {result_knn['accuracy']:.4f}")
print(f"  Precision: {result_knn['precision']:.4f}")
print(f"  Recall: {result_knn['recall']:.4f}")
print(f"  F1: {result_knn['f1']:.4f}")
print(f"  ROC-AUC: {result_knn['roc_auc']:.4f}")
print(f"  Total Revenue: €{result_knn['total_revenue']:.2f}")
print(f"  Avg Revenue: €{result_knn['avg_revenue']:.4f}")

# Confusion matrix plot
plot_confusion_matrix(
    y_val, result_knn['y_pred_optimal'],
    f"KNN (threshold={result_knn['best_threshold']:.4f})"
)
plt.show()

# Classification report
print("\nClassification Report:")
print(result_knn['classification_report'])


### 5.6 XGBoost or HistGradientBoosting


In [ ]:
if XGBOOST_AVAILABLE:
    print("Training XGBoost...")
    print("=" * 80)
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        min_child_weight=1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        eval_metric='logloss'
    )
    
    result_xgb, model_xgb = evaluate_model(xgb_model, 'XGBoost', X_train_df, y_train, X_val_df, y_val)
    models['XGBoost'] = model_xgb
    results.append(result_xgb)
    
    print(f"\nValidation Metrics (at threshold={result_xgb['best_threshold']:.4f}):")
    print(f"  Accuracy: {result_xgb['accuracy']:.4f}")
    print(f"  Precision: {result_xgb['precision']:.4f}")
    print(f"  Recall: {result_xgb['recall']:.4f}")
    print(f"  F1: {result_xgb['f1']:.4f}")
    print(f"  ROC-AUC: {result_xgb['roc_auc']:.4f}")
    print(f"  Total Revenue: €{result_xgb['total_revenue']:.2f}")
    print(f"  Avg Revenue: €{result_xgb['avg_revenue']:.4f}")
    
    # Confusion matrix plot
    plot_confusion_matrix(
        y_val, result_xgb['y_pred_optimal'],
        f"XGBoost (threshold={result_xgb['best_threshold']:.4f})"
    )
    plt.show()
    
    # Classification report
    print("\nClassification Report:")
    print(result_xgb['classification_report'])
else:
    print("Training HistGradientBoostingClassifier (XGBoost fallback)...")
    print("=" * 80)
    hgb = HistGradientBoostingClassifier(
        max_iter=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=RANDOM_STATE
    )
    
    result_hgb, model_hgb = evaluate_model(hgb, 'HistGradientBoosting', X_train_df, y_train, X_val_df, y_val)
    models['HistGradientBoosting'] = model_hgb
    results.append(result_hgb)
    
    print(f"\nValidation Metrics (at threshold={result_hgb['best_threshold']:.4f}):")
    print(f"  Accuracy: {result_hgb['accuracy']:.4f}")
    print(f"  Precision: {result_hgb['precision']:.4f}")
    print(f"  Recall: {result_hgb['recall']:.4f}")
    print(f"  F1: {result_hgb['f1']:.4f}")
    print(f"  ROC-AUC: {result_hgb['roc_auc']:.4f}")
    print(f"  Total Revenue: €{result_hgb['total_revenue']:.2f}")
    print(f"  Avg Revenue: €{result_hgb['avg_revenue']:.4f}")
    
    # Confusion matrix plot
    plot_confusion_matrix(
        y_val, result_hgb['y_pred_optimal'],
        f"HistGradientBoosting (threshold={result_hgb['best_threshold']:.4f})"
    )
    plt.show()
    
    # Classification report
    print("\nClassification Report:")
    print(result_hgb['classification_report'])


### 5.7 Hyperparameter Tuning with Optuna

Tune hyperparameters for a strong model (XGBoost or Random Forest) using Optuna, optimizing for **validation revenue**.


In [ ]:
if OPTUNA_AVAILABLE:
    print("Setting up Optuna hyperparameter tuning...")
    
    # Choose model to tune (prefer XGBoost if available, else Random Forest)
    if XGBOOST_AVAILABLE:
        model_to_tune = 'XGBoost'
        
        def objective_xgb(trial):
            """Optuna objective for XGBoost, maximizing validation revenue."""
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'random_state': RANDOM_STATE,
                'n_jobs': -1,
                'eval_metric': 'logloss'
            }
            
            model = xgb.XGBClassifier(**params)
            model.fit(X_train_df, y_train)
            
            # Get probabilities
            y_pred_proba = model.predict_proba(X_val_df)[:, 1]
            
            # Optimize threshold to maximize revenue
            best_threshold, best_revenue, _ = optimize_threshold(y_val, y_pred_proba)
            
            return best_revenue  # Optuna maximizes, so return revenue directly
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE)
        )
        
        print(f"Running Optuna optimization for {model_to_tune} (maximizing validation revenue)...")
        print("This may take several minutes...")
        
        study.optimize(objective_xgb, n_trials=60, show_progress_bar=True)
        
        print(f"\nBest trial revenue: €{study.best_value:.2f}")
        print(f"Best parameters: {study.best_params}")
        
        # Train final model with best parameters
        best_params = study.best_params.copy()
        best_params['random_state'] = RANDOM_STATE
        best_params['n_jobs'] = -1
        best_params['eval_metric'] = 'logloss'
        
        xgb_tuned = xgb.XGBClassifier(**best_params)
        result_xgb_tuned, model_xgb_tuned = evaluate_model(
            xgb_tuned, 'XGBoost (Optuna Tuned)', X_train_df, y_train, X_val_df, y_val
        )
        models['XGBoost (Optuna Tuned)'] = model_xgb_tuned
        results.append(result_xgb_tuned)
        
        print(f"\nTuned Model Performance:")
        print(f"  Accuracy: {result_xgb_tuned['accuracy']:.4f}")
        print(f"  Precision: {result_xgb_tuned['precision']:.4f}")
        print(f"  Recall: {result_xgb_tuned['recall']:.4f}")
        print(f"  F1: {result_xgb_tuned['f1']:.4f}")
        print(f"  ROC-AUC: {result_xgb_tuned['roc_auc']:.4f}")
        print(f"  Best Threshold: {result_xgb_tuned['best_threshold']:.4f}")
        print(f"  Total Revenue: €{result_xgb_tuned['total_revenue']:.2f}")
        print(f"  Avg Revenue: €{result_xgb_tuned['avg_revenue']:.4f}")
        
        # Confusion matrix plot
        plot_confusion_matrix(
            y_val, result_xgb_tuned['y_pred_optimal'],
            f"XGBoost (Optuna Tuned, threshold={result_xgb_tuned['best_threshold']:.4f})"
        )
        plt.show()
        
        # Classification report
        print("\nClassification Report:")
        print(result_xgb_tuned['classification_report'])
        
    else:
        # Tune Random Forest
        model_to_tune = 'Random Forest'
        
        def objective_rf(trial):
            """Optuna objective for Random Forest, maximizing validation revenue."""
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'max_depth': trial.suggest_int('max_depth', 5, 20),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'class_weight': 'balanced',
                'random_state': RANDOM_STATE,
                'n_jobs': -1
            }
            
            model = RandomForestClassifier(**params)
            model.fit(X_train_df, y_train)
            
            y_pred_proba = model.predict_proba(X_val_df)[:, 1]
            best_threshold, best_revenue, _ = optimize_threshold(y_val, y_pred_proba)
            
            return best_revenue
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE)
        )
        
        print(f"Running Optuna optimization for {model_to_tune} (maximizing validation revenue)...")
        print("This may take several minutes...")
        
        study.optimize(objective_rf, n_trials=60, show_progress_bar=True)
        
        print(f"\nBest trial revenue: €{study.best_value:.2f}")
        print(f"Best parameters: {study.best_params}")
        
        # Train final model
        best_params = study.best_params.copy()
        best_params['random_state'] = RANDOM_STATE
        best_params['n_jobs'] = -1
        
        rf_tuned = RandomForestClassifier(**best_params)
        result_rf_tuned, model_rf_tuned = evaluate_model(
            rf_tuned, 'Random Forest (Optuna Tuned)', X_train_df, y_train, X_val_df, y_val
        )
        models['Random Forest (Optuna Tuned)'] = model_rf_tuned
        results.append(result_rf_tuned)
        
        print(f"\nTuned Model Performance:")
        print(f"  Accuracy: {result_rf_tuned['accuracy']:.4f}")
        print(f"  Precision: {result_rf_tuned['precision']:.4f}")
        print(f"  Recall: {result_rf_tuned['recall']:.4f}")
        print(f"  F1: {result_rf_tuned['f1']:.4f}")
        print(f"  ROC-AUC: {result_rf_tuned['roc_auc']:.4f}")
        print(f"  Best Threshold: {result_rf_tuned['best_threshold']:.4f}")
        print(f"  Total Revenue: €{result_rf_tuned['total_revenue']:.2f}")
        print(f"  Avg Revenue: €{result_rf_tuned['avg_revenue']:.4f}")
        
        # Confusion matrix plot
        plot_confusion_matrix(
            y_val, result_rf_tuned['y_pred_optimal'],
            f"Random Forest (Optuna Tuned, threshold={result_rf_tuned['best_threshold']:.4f})"
        )
        plt.show()
        
        # Classification report
        print("\nClassification Report:")
        print(result_rf_tuned['classification_report'])
else:
    print("Optuna not available. Skipping hyperparameter tuning.")


## 6. Threshold Optimization

Threshold optimization is performed during model evaluation. We search over a range of thresholds to find the one that maximizes validation revenue, since the default 0.5 threshold is unlikely to be optimal for our asymmetric cost structure. Here we visualize the threshold-revenue relationship for the best model.


In [ ]:
# Find best model by revenue
best_model_result = max(results, key=lambda x: x['total_revenue'])
best_model_name = best_model_result['model_name']

print(f"Best model by revenue: {best_model_name}")
print(f"Best threshold: {best_model_result['best_threshold']:.4f}")
print(f"Total revenue: €{best_model_result['total_revenue']:.2f}")
print(f"Average revenue: €{best_model_result['avg_revenue']:.4f}")

# Plot threshold-revenue curve for best model
threshold_df = best_model_result['threshold_results']

plt.figure(figsize=(10, 6))
plt.plot(threshold_df['threshold'], threshold_df['total_revenue'], 'b-', linewidth=2)
plt.axvline(best_model_result['best_threshold'], color='r', linestyle='--', 
            label=f'Optimal threshold = {best_model_result["best_threshold"]:.4f}')
plt.xlabel('Threshold (send voucher if p < threshold)')
plt.ylabel('Total Revenue (€)')
plt.title(f'Revenue vs Threshold: {best_model_name}')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


## 7. Model Comparison Table


In [ ]:
# Create comparison DataFrame
comparison_data = []
for r in results:
    comparison_data.append({
        'Model': r['model_name'],
        'Accuracy': r['accuracy'],
        'Precision': r['precision'],
        'Recall': r['recall'],
        'F1': r['f1'],
        'ROC-AUC': r['roc_auc'],
        'Best Threshold': r['best_threshold'],
        'Total Revenue (€)': r['total_revenue'],
        'Avg Revenue (€)': r['avg_revenue']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Total Revenue (€)', ascending=False)

print("Model Comparison (sorted by Total Revenue):")
print("=" * 100)
print(comparison_df.to_string(index=False))
print("=" * 100)

# Save comparison table
comparison_df.to_csv('model_comparison.csv', index=False)
print("\nComparison table saved to 'model_comparison.csv'")

# Model comparison bar chart: Total Revenue by Model
plt.figure(figsize=(12, 6))
colors = plt.cm.viridis(np.linspace(0, 1, len(comparison_df)))
bars = plt.barh(comparison_df['Model'], comparison_df['Total Revenue (€)'], color=colors)
plt.xlabel('Total Revenue (€)', fontsize=12)
plt.ylabel('Model', fontsize=12)
plt.title('Model Comparison: Total Validation Revenue', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars, comparison_df['Total Revenue (€)'])):
    plt.text(value + max(comparison_df['Total Revenue (€)']) * 0.01, bar.get_y() + bar.get_height()/2,
             f'€{value:.2f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# Calibration curves for top 2 models
print("\nCalibration Curves (Top 2 Models by Revenue):")
print("=" * 80)

top_2_models = comparison_df.head(2)['Model'].tolist()
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, model_name in enumerate(top_2_models):
    # Find the result for this model
    model_result = next(r for r in results if r['model_name'] == model_name)
    y_pred_proba = model_result['y_pred_proba']
    
    # Compute calibration curve
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_val, y_pred_proba, n_bins=10, strategy='uniform'
    )
    
    ax = axes[idx]
    ax.plot(mean_predicted_value, fraction_of_positives, "s-", label=model_name)
    ax.plot([0, 1], [0, 1], "k--", label="Perfectly calibrated")
    ax.set_xlabel('Mean Predicted Probability', fontsize=11)
    ax.set_ylabel('Fraction of Positives', fontsize=11)
    ax.set_title(f'Calibration Curve: {model_name}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: Calibration curves show how well predicted probabilities match actual frequencies.")
print("A well-calibrated model should lie close to the diagonal line.")


## 8. Final Model Selection

Select the best model based on validation revenue.


In [ ]:
# Select best model
best_model_name = best_model_result['model_name']
best_model = models[best_model_name]
best_threshold = best_model_result['best_threshold']

print(f"Selected Best Model: {best_model_name}")
print(f"Optimal Threshold: {best_threshold:.4f}")
print(f"Validation Total Revenue: €{best_model_result['total_revenue']:.2f}")
print(f"Validation Average Revenue: €{best_model_result['avg_revenue']:.4f}")
print(f"Validation Accuracy: {best_model_result['accuracy']:.4f}")
print(f"Validation ROC-AUC: {best_model_result['roc_auc']:.4f}")

# Print voucher send-rate on validation
val_send_rate = best_model_result['y_pred_optimal'].mean()
print(f"\nVoucher Send-Rate on Validation: {val_send_rate*100:.2f}%")
print(f"  ({best_model_result['y_pred_optimal'].sum()} vouchers sent out of {len(best_model_result['y_pred_optimal'])} customers)")

# Save best model and threshold
joblib.dump(best_model, 'best_model.joblib')
print("\nBest model saved to 'best_model.joblib'")

with open('best_threshold.json', 'w') as f:
    json.dump({'threshold': float(best_threshold), 'model_name': best_model_name}, f, indent=2)
print("Best threshold saved to 'best_threshold.json'")


## 9. Predict on Unlabeled Test Set and Export Submission

Train the final model on the full labeled data (train + validation) and generate predictions for the test set. The test set (test.csv) is unlabeled, so we cannot compute out-of-sample metrics. All evaluation and model selection must be based on the validation set.


In [ ]:
# Combine train and validation for final training
X_full = pd.concat([X_train_df, X_val_df], axis=0)
y_full = pd.concat([y_train, y_val], axis=0)

print(f"Full training set shape: {X_full.shape}")
print(f"Full training target distribution: {y_full.value_counts().to_dict()}")

# Retrain best model on full data
print(f"\nRetraining {best_model_name} on full labeled data...")

# Get the model class and parameters
if 'XGBoost' in best_model_name:
    if hasattr(best_model, 'get_params'):
        final_model = xgb.XGBClassifier(**best_model.get_params())
    else:
        # Fallback: use default parameters
        final_model = xgb.XGBClassifier(random_state=RANDOM_STATE, n_jobs=-1)
elif 'Random Forest' in best_model_name:
    if hasattr(best_model, 'get_params'):
        final_model = RandomForestClassifier(**best_model.get_params())
    else:
        final_model = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
elif 'Gradient Boosting' in best_model_name:
    if hasattr(best_model, 'get_params'):
        final_model = GradientBoostingClassifier(**best_model.get_params())
    else:
        final_model = GradientBoostingClassifier(random_state=RANDOM_STATE)
elif 'AdaBoost' in best_model_name:
    if hasattr(best_model, 'get_params'):
        final_model = AdaBoostClassifier(**best_model.get_params())
    else:
        final_model = AdaBoostClassifier(random_state=RANDOM_STATE)
elif 'KNN' in best_model_name:
    if hasattr(best_model, 'get_params'):
        # Clone the pipeline to preserve tuned parameters
        from sklearn.base import clone
        final_model = clone(best_model)
    else:
        final_model = Pipeline([
            ('scaler', StandardScaler()),
            ('knn', KNeighborsClassifier(n_neighbors=5, weights='distance'))
        ])
elif 'Logistic' in best_model_name:
    if hasattr(best_model, 'get_params'):
        final_model = LogisticRegression(**best_model.get_params())
    else:
        final_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
elif 'HistGradientBoosting' in best_model_name:
    if hasattr(best_model, 'get_params'):
        final_model = HistGradientBoostingClassifier(**best_model.get_params())
    else:
        final_model = HistGradientBoostingClassifier(random_state=RANDOM_STATE)
else:
    # Generic fallback: clone the model
    from sklearn.base import clone
    final_model = clone(best_model)

final_model.fit(X_full, y_full)
print("Model retrained on full data.")

# Predict on test set
print("\nGenerating predictions on test set...")
p_test = final_model.predict_proba(X_test_df)[:, 1]

# Ensure probabilities are valid
assert np.all((p_test >= 0) & (p_test <= 1)), "Invalid probabilities detected"

# Apply decision rule: send_voucher = 1 if p < threshold
voucher_decision = (p_test < best_threshold).astype(int)

print(f"Test set predictions generated.")
print(f"Number of vouchers to send: {voucher_decision.sum()} out of {len(voucher_decision)} ({voucher_decision.mean()*100:.2f}%)")

# Quality checks
print("\nQuality Checks:")
print(f"  Probabilities in valid range [0,1]: {np.all((p_test >= 0) & (p_test <= 1))}")
print(f"  Voucher decisions are binary: {set(voucher_decision) <= {0, 1}}")
print(f"  Test set voucher send-rate: {voucher_decision.mean()*100:.2f}%")

# Create submission DataFrame
# Note: If customernumber is not available in test, we'll use index
try:
    # Try to load test.csv to get customernumber if available
    test_raw = pd.read_csv('test.csv', sep=';', low_memory=False)
    if 'customernumber' in test_raw.columns:
        submission = pd.DataFrame({
            'customernumber': test_raw['customernumber'].values,
            'voucher_decision': voucher_decision
        })
    else:
        submission = pd.DataFrame({
            'customernumber': X_test_df.index,
            'voucher_decision': voucher_decision
        })
except:
    submission = pd.DataFrame({
        'customernumber': X_test_df.index,
        'voucher_decision': voucher_decision
    })

# Export submission
submission.to_csv('submission.csv', index=False)
print(f"\nSubmission file saved to 'submission.csv'")
print(f"Submission shape: {submission.shape}")
print(f"\nFirst few rows:")
print(submission.head(10))


## 10. Interpretability Hooks

Extract feature importances and coefficients for model interpretation.


In [ ]:
# Feature importances for tree-based models
print("Feature Importances (Tree-based Models):")
print("=" * 80)

for model_name, model in models.items():
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        feature_imp_df = pd.DataFrame({
            'feature': X_train_df.columns,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print(f"\n{model_name} - Top 15 Features:")
        print(feature_imp_df.head(15).to_string(index=False))
        
        # Save to CSV
        safe_name = model_name.replace(' ', '_').replace('(', '').replace(')', '')
        feature_imp_df.to_csv(f'feature_importance_{safe_name}.csv', index=False)

print("\n" + "=" * 80)


In [ ]:
# Coefficients for Logistic Regression
if 'Logistic Regression' in models:
    lr_model = models['Logistic Regression']
    if hasattr(lr_model, 'coef_'):
        coef_df = pd.DataFrame({
            'feature': X_train_df.columns,
            'coefficient': lr_model.coef_[0]
        }).sort_values('coefficient', key=abs, ascending=False)
        
        print("Logistic Regression - Top 20 Coefficients (by absolute value):")
        print("=" * 80)
        print(coef_df.head(20).to_string(index=False))
        
        coef_df.to_csv('logistic_regression_coefficients.csv', index=False)
        print("\nCoefficients saved to 'logistic_regression_coefficients.csv'")


In [ ]:
# SHAP analysis placeholder
print("SHAP Analysis Setup:")
print("=" * 80)
print("\nTo perform SHAP analysis, install shap and run:")
print("\n```python")
print("import shap")
print("\n# For tree models:")
print(f"explainer = shap.TreeExplainer({best_model_name})")
print(f"shap_values = explainer.shap_values(X_val_df.iloc[:100])  # Sample for speed")
print(f"shap.summary_plot(shap_values, X_val_df.iloc[:100])")
print("\n# For other models:")
print(f"explainer = shap.KernelExplainer({best_model_name}.predict_proba, X_val_df.iloc[:100])")
print(f"shap_values = explainer.shap_values(X_val_df.iloc[:100])")
print("```")
print("\nNote: Store the trained model and feature matrix for SHAP analysis.")

# Save model and data for SHAP
shap_data = {
    'model': best_model,
    'X_val_sample': X_val_df.iloc[:100].values,  # Sample for SHAP
    'feature_names': X_train_df.columns.tolist()
}

joblib.dump(shap_data, 'shap_data.joblib')
print("\nModel and sample data saved to 'shap_data.joblib' for SHAP analysis.")


## Summary

This notebook has:
1. ✅ Loaded and validated preprocessed data
2. ✅ Implemented revenue calculation and threshold optimization
3. ✅ Trained multiple models (Logistic Regression, Random Forest, Gradient Boosting, KNN, XGBoost/HistGB)
4. ✅ Performed hyperparameter tuning with Optuna (optimizing for revenue)
5. ✅ Selected best model based on validation revenue
6. ✅ Generated test predictions and submission file
7. ✅ Extracted feature importances and interpretability hooks

**Key Outputs:**
- `best_model.joblib`: Trained best model
- `best_threshold.json`: Optimal threshold
- `submission.csv`: Test predictions
- `model_comparison.csv`: Comparison of all models
- `feature_importance_*.csv`: Feature importances for tree models
- `logistic_regression_coefficients.csv`: Coefficients for logistic regression
